# CAPSTONE PROJECT - STARBUCKS
# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# About this Notebook
The code in this notebook was developed using Amazon Sage-Maker notebook instances and it uses several AWS services, such as Sage-Maker Endpoints, Models and S3 buckets. This notebook will not run outside this environment.

## Load Data

In [1]:
import pandas as pd
import numpy as np
import math
import json
import seaborn as sns
import boto3
import sagemaker
import os
from sklearn.model_selection import train_test_split

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

No handlers could be found for logger "sagemaker"


## AWS Services initialization

In [2]:
# session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# create an S3 bucket
bucket = sagemaker_session.default_bucket()

## Take a look at the datasets

### 1. The Portfolio dataset describes the several offers made to customers

In [3]:
portfolio.head()

,channels,difficulty,duration,id,offer_type,reward
0,"[email, mobile, social]",10,7,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10
1,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10
2,"[web, email, mobile]",0,4,3f207df678b143eea3cee63160fa8bed,informational,0
3,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5
4,"[web, email]",20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,5


### 2. The Profile dataset describes every costumer

In [4]:
profile.head()

,age,became_member_on,gender,id,income
0,118,20170212,None,68be06ca386d4c31939f3a4f0e3dd783,NaN
1,55,20170715,F,0610b486422d4921ae7d2bf64640c50b,112000.0
2,118,20180712,None,38fe809add3b4fcf9315a9694bb96ff5,NaN
3,75,20170509,F,78afa995795e4d85b5d9ceeca43f5fef,100000.0
4,118,20170804,None,a03223e636434f42ac4c3df47e8bac43,NaN


### 3. The Transcript dataset describes every transaction and offer action made by the several customers 

In [5]:
transcript.head()

,event,person,time,value
0,offer received,78afa995795e4d85b5d9ceeca43f5fef,0,{u'offer id': u'9b98b8c7a33c4b65b9aebfe6a799e6...
1,offer received,a03223e636434f42ac4c3df47e8bac43,0,{u'offer id': u'0b1e1539f2cc45b7b9fa7c272da2e1...
2,offer received,e2127556f4f64592b11af22de27a7932,0,{u'offer id': u'2906b810c7d4411798c6938adc9daa...
3,offer received,8ec6ce2a7e7949b1bf142def7d0e0586,0,{u'offer id': u'fafdcd668e3743c1bb461111dcafc2...
4,offer received,68617ca6246f4fbc85e91a2a49552598,0,{u'offer id': u'4d5c57ea9a6940dd891ad53e9dbe8d...


# 1. Pre Processing
___
## 1.1 Data Cleaning

___
Remove invalid customers by removing all rows with NaN values in the *income* feature
___

In [6]:
profile = profile.dropna(subset=['income'])

In [7]:
profile.head()

,age,became_member_on,gender,id,income
1,55,20170715,F,0610b486422d4921ae7d2bf64640c50b,112000.0
3,75,20170509,F,78afa995795e4d85b5d9ceeca43f5fef,100000.0
5,68,20180426,M,e2127556f4f64592b11af22de27a7932,70000.0
8,65,20180209,M,389bc3fa690240e798340f5a15918d5c,53000.0
12,58,20171111,M,2eeac8d8feae4a8cad5a6af0499a211d,51000.0


## 1.2 Feature Engineering

___
One Hot Encode the *channels* feature in the portfolio dataset
___

In [8]:
portfolio['channels'] = portfolio['channels'].apply(lambda x: str([item.encode('utf-8') for item in x]).replace(' ','').replace('[','').replace(']','').replace("'",""))
portfolio.head()

,channels,difficulty,duration,id,offer_type,reward
0,"email,mobile,social",10,7,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10
1,"web,email,mobile,social",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10
2,"web,email,mobile",0,4,3f207df678b143eea3cee63160fa8bed,informational,0
3,"web,email,mobile",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5
4,"web,email",20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,5


In [9]:
dumdum = portfolio.channels.str.split('\s*,\s*', expand=True).stack().str.get_dummies().sum(level=0)

In [10]:
portfolio[['mobile','social','web']] = dumdum.reset_index()[['mobile','social','web']]

In [11]:
portfolio = portfolio.drop(columns = ['channels'])
portfolio.head()

,difficulty,duration,id,offer_type,reward,mobile,social,web
0,10,7,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10,1,1,0
1,10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10,1,1,1
2,0,4,3f207df678b143eea3cee63160fa8bed,informational,0,1,0,1
3,5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5,1,0,1
4,20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,5,0,0,1


___
Transform feature *became_member_on* to the number of days of membership, counting from the year the first member joined starcbucks
___
In this case, the first member joined in 2013
___

In [12]:
profile.became_member_on.min() / 10000

2013

In [13]:
first_member = int(profile.became_member_on.min() / 10000) * 10000

In [14]:
profile.became_member_on = profile.became_member_on - first_member

In [15]:
profile.head()

,age,became_member_on,gender,id,income
1,55,40715,F,0610b486422d4921ae7d2bf64640c50b,112000.0
3,75,40509,F,78afa995795e4d85b5d9ceeca43f5fef,100000.0
5,68,50426,M,e2127556f4f64592b11af22de27a7932,70000.0
8,65,50209,M,389bc3fa690240e798340f5a15918d5c,53000.0
12,58,41111,M,2eeac8d8feae4a8cad5a6af0499a211d,51000.0


In [16]:
profile['days_by_year'] = (profile['became_member_on'] /(10000)).astype(int)*365 
profile.became_member_on = profile['became_member_on'].apply(lambda x: x - (int(x/10000) * 10000) if x >= 10000 else x) 
profile['days_by_month'] = (profile['became_member_on'] /(100)).astype(int)*30 
profile.became_member_on = profile['became_member_on'].apply(lambda x: x - (int(x/100) * 100) if x >= 100 else x) 
profile.head()

,age,became_member_on,gender,id,income,days_by_year,days_by_month
1,55,15,F,0610b486422d4921ae7d2bf64640c50b,112000.0,1460,210
3,75,9,F,78afa995795e4d85b5d9ceeca43f5fef,100000.0,1460,150
5,68,26,M,e2127556f4f64592b11af22de27a7932,70000.0,1825,120
8,65,9,M,389bc3fa690240e798340f5a15918d5c,53000.0,1825,60
12,58,11,M,2eeac8d8feae4a8cad5a6af0499a211d,51000.0,1460,330


In [17]:
profile['days'] = profile['became_member_on'] + profile['days_by_year'] + profile['days_by_month']
profile = profile.drop(columns = ['became_member_on','days_by_year','days_by_month'])

In [18]:
profile.head()

,age,gender,id,income,days
1,55,F,0610b486422d4921ae7d2bf64640c50b,112000.0,1685
3,75,F,78afa995795e4d85b5d9ceeca43f5fef,100000.0,1619
5,68,M,e2127556f4f64592b11af22de27a7932,70000.0,1971
8,65,M,389bc3fa690240e798340f5a15918d5c,53000.0,1894
12,58,M,2eeac8d8feae4a8cad5a6af0499a211d,51000.0,1801


### Transcript

___
Get the information out of the *value* feature
___

In [19]:
transcript.event.unique()

array([u'offer received', u'offer viewed', u'transaction',
       u'offer completed'], dtype=object)

In [20]:
transcript[transcript.event == 'offer completed'].head()

,event,person,time,value
12658,offer completed,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0,{u'offer_id': u'2906b810c7d4411798c6938adc9daa...
12672,offer completed,fe97aa22dd3e48c8b143116a8403dd52,0,{u'offer_id': u'fafdcd668e3743c1bb461111dcafc2...
12679,offer completed,629fc02d56414d91bca360decdfa9288,0,{u'offer_id': u'9b98b8c7a33c4b65b9aebfe6a799e6...
12692,offer completed,676506bad68e4161b9bbaffeb039626b,0,{u'offer_id': u'ae264e3637204a6fb9bb56bc8210dd...
12697,offer completed,8f7dd3b2afe14c078eb4f6e6fe4ba97d,0,{u'offer_id': u'4d5c57ea9a6940dd891ad53e9dbe8d...


In [21]:
transactions = transcript[transcript.event == 'transaction']

In [22]:
transactions.head()

,event,person,time,value
12654,transaction,02c083884c7d45b39cc68e1314fec56c,0,{u'amount': 0.83}
12657,transaction,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0,{u'amount': 34.56}
12659,transaction,54890f68699049c2a04d415abc25e717,0,{u'amount': 13.23}
12670,transaction,b2f1cd155b864803ad8334cdf13c4bd2,0,{u'amount': 19.51}
12671,transaction,fe97aa22dd3e48c8b143116a8403dd52,0,{u'amount': 18.97}


In [23]:
offer_events =  transcript[~(transcript.event == 'transaction')]

In [24]:
offer_events.head()

,event,person,time,value
0,offer received,78afa995795e4d85b5d9ceeca43f5fef,0,{u'offer id': u'9b98b8c7a33c4b65b9aebfe6a799e6...
1,offer received,a03223e636434f42ac4c3df47e8bac43,0,{u'offer id': u'0b1e1539f2cc45b7b9fa7c272da2e1...
2,offer received,e2127556f4f64592b11af22de27a7932,0,{u'offer id': u'2906b810c7d4411798c6938adc9daa...
3,offer received,8ec6ce2a7e7949b1bf142def7d0e0586,0,{u'offer id': u'fafdcd668e3743c1bb461111dcafc2...
4,offer received,68617ca6246f4fbc85e91a2a49552598,0,{u'offer id': u'4d5c57ea9a6940dd891ad53e9dbe8d...


___
Get the offer id from transactions that correspond to interactions with offers ( **offer received, offer viewed, offer completed**) from the *value* feature
___

In [25]:
offer_events['offer'] = [d.get('offer id') if d.get('offer id') is not None else (d.get('offer_id')) for d in offer_events.value]

/home/ec2-user/anaconda3/envs/amazonei_mxnet_p27/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [26]:
offer_events = offer_events.drop(columns = ['value'])
offer_events.head()

,event,person,time,offer
0,offer received,78afa995795e4d85b5d9ceeca43f5fef,0,9b98b8c7a33c4b65b9aebfe6a799e6d9
1,offer received,a03223e636434f42ac4c3df47e8bac43,0,0b1e1539f2cc45b7b9fa7c272da2e1d7
2,offer received,e2127556f4f64592b11af22de27a7932,0,2906b810c7d4411798c6938adc9daaa5
3,offer received,8ec6ce2a7e7949b1bf142def7d0e0586,0,fafdcd668e3743c1bb461111dcafc2a4
4,offer received,68617ca6246f4fbc85e91a2a49552598,0,4d5c57ea9a6940dd891ad53e9dbe8da0


___
Get the value from simple transactions (**transaction** event)
___

In [27]:
transactions['value'] = [d.get('amount') for d in transactions.value]

/home/ec2-user/anaconda3/envs/amazonei_mxnet_p27/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [28]:
transactions.head()

,event,person,time,value
12654,transaction,02c083884c7d45b39cc68e1314fec56c,0,0.83
12657,transaction,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0,34.56
12659,transaction,54890f68699049c2a04d415abc25e717,0,13.23
12670,transaction,b2f1cd155b864803ad8334cdf13c4bd2,0,19.51
12671,transaction,fe97aa22dd3e48c8b143116a8403dd52,0,18.97


___
Compute the customers lifetime value by calculating the monetary, frequency and recency features for each of them
___
- monetary: total ammount spent
- frequency: frequency of transactions
- recency: how recent was the last transaction
___

In [29]:
monetary_frequency = transactions.groupby('person').value.agg(['sum','count']).rename(columns = {'sum': 'monetary','count': 'frequency'})
monetary_frequency.head()

,monetary,frequency
person,,
0009655768c64bdeb2e877511632db8f,127.60,8
00116118485d4dfda04fdbaba9a87b5c,4.09,3
0011e0d4e6b944f998e987f904e8c1e5,79.46,5
0020c2b971eb4e9188eac86d93036a77,196.86,8
0020ccbbb6d84e358d3414a3ff76cffd,154.05,12


In [30]:
time_max = transactions.time.max()
recency = transactions.groupby('person').time.max().reset_index().rename(columns = {'time': 'recency'})
recency['recency'] = time_max - recency['recency']
recency.head()

,person,recency
0,0009655768c64bdeb2e877511632db8f,18
1,00116118485d4dfda04fdbaba9a87b5c,240
2,0011e0d4e6b944f998e987f904e8c1e5,60
3,0020c2b971eb4e9188eac86d93036a77,6
4,0020ccbbb6d84e358d3414a3ff76cffd,42


In [31]:
rfm = transactions.merge(monetary_frequency, right_on='person',left_on = 'person')
rfm = rfm.merge(recency,right_on='person',left_on = 'person')
rfm = rfm[['person','recency','frequency','monetary']].drop_duplicates()
rfm = rfm.merge(profile, right_on='id',left_on='person')
rfm.head()

,person,recency,frequency,monetary,age,gender,id,income,days
0,02c083884c7d45b39cc68e1314fec56c,294,10,29.89,20,F,02c083884c7d45b39cc68e1314fec56c,30000.0,1316
1,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,174,12,320.48,42,M,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,96000.0,1142
2,54890f68699049c2a04d415abc25e717,30,7,118.85,36,M,54890f68699049c2a04d415abc25e717,56000.0,1848
3,b2f1cd155b864803ad8334cdf13c4bd2,144,8,195.35,55,F,b2f1cd155b864803ad8334cdf13c4bd2,94000.0,1776
4,fe97aa22dd3e48c8b143116a8403dd52,6,11,562.77,39,F,fe97aa22dd3e48c8b143116a8403dd52,67000.0,1837


___
Create a matrix *person x offer* , which states if the offer was **received, viewed and completed**
___

In [32]:
offer_events.head()

,event,person,time,offer
0,offer received,78afa995795e4d85b5d9ceeca43f5fef,0,9b98b8c7a33c4b65b9aebfe6a799e6d9
1,offer received,a03223e636434f42ac4c3df47e8bac43,0,0b1e1539f2cc45b7b9fa7c272da2e1d7
2,offer received,e2127556f4f64592b11af22de27a7932,0,2906b810c7d4411798c6938adc9daaa5
3,offer received,8ec6ce2a7e7949b1bf142def7d0e0586,0,fafdcd668e3743c1bb461111dcafc2a4
4,offer received,68617ca6246f4fbc85e91a2a49552598,0,4d5c57ea9a6940dd891ad53e9dbe8da0


In [33]:
offer_events = offer_events[['person','event','offer']].drop_duplicates()
offer_events['val'] = int(1)

In [34]:
offer_events['person_offer'] = offer_events['person'] + ':' + offer_events['offer']

In [35]:
offer_acceptance = offer_events.pivot(index = 'person_offer',columns = 'event',values = 'val').reset_index().fillna(0)

In [36]:
offer_acceptance['person'] = offer_acceptance.apply(lambda x : x['person_offer'].split(':')[0],axis=1)
offer_acceptance['offer'] = offer_acceptance.apply(lambda x : x['person_offer'].split(':')[1],axis=1)
offer_acceptance = offer_acceptance.drop(columns = ['person_offer'])
offer_acceptance.head()

event,offer completed,offer received,offer viewed,person,offer
0,1.0,1.0,0.0,0009655768c64bdeb2e877511632db8f,2906b810c7d4411798c6938adc9daaa5
1,0.0,1.0,1.0,0009655768c64bdeb2e877511632db8f,3f207df678b143eea3cee63160fa8bed
2,0.0,1.0,1.0,0009655768c64bdeb2e877511632db8f,5a8bc65990b245e5a138643cd4eb9837
3,1.0,1.0,1.0,0009655768c64bdeb2e877511632db8f,f19421c1d4aa40978ebb69ca19b0e20d
4,1.0,1.0,1.0,0009655768c64bdeb2e877511632db8f,fafdcd668e3743c1bb461111dcafc2a4


## 2. Data Preparation

### 2.1 Combine all the information into one dataframe


___
Add info (**rfm, income, days, age, gender**) about the costumers to the previously computed matrix
___

In [37]:
data = rfm.merge(offer_acceptance,left_on = 'person',right_on = 'person')
data.head()

,person,recency,frequency,monetary,age,gender,id,income,days,offer completed,offer received,offer viewed,offer
0,02c083884c7d45b39cc68e1314fec56c,294,10,29.89,20,F,02c083884c7d45b39cc68e1314fec56c,30000.0,1316,0.0,1.0,0.0,0b1e1539f2cc45b7b9fa7c272da2e1d7
1,02c083884c7d45b39cc68e1314fec56c,294,10,29.89,20,F,02c083884c7d45b39cc68e1314fec56c,30000.0,1316,0.0,1.0,1.0,ae264e3637204a6fb9bb56bc8210ddfd
2,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,174,12,320.48,42,M,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,96000.0,1142,1.0,1.0,1.0,0b1e1539f2cc45b7b9fa7c272da2e1d7
3,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,174,12,320.48,42,M,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,96000.0,1142,1.0,1.0,1.0,2298d6c36e964ae4a3e7e9706d1fb8c2
4,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,174,12,320.48,42,M,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,96000.0,1142,1.0,1.0,1.0,2906b810c7d4411798c6938adc9daaa5


___
One-Hot-Encode the customer *gender*
___

In [38]:
data_gender = pd.get_dummies(data.gender, prefix='gender')

In [39]:
data = pd.concat([data, data_gender], axis=1).drop(columns = ['gender','person','id'])

In [40]:
data = data.merge(portfolio, left_on = 'offer', right_on = 'id').drop(columns = ['offer','id'])

In [41]:
data.head()

,recency,frequency,monetary,age,income,days,offer completed,offer received,offer viewed,gender_F,gender_M,gender_O,difficulty,duration,offer_type,reward,mobile,social,web
0,294,10,29.89,20,30000.0,1316,0.0,1.0,0.0,1,0,0,20,10,discount,5,0,0,1
1,174,12,320.48,42,96000.0,1142,1.0,1.0,1.0,0,1,0,20,10,discount,5,0,0,1
2,30,7,118.85,36,56000.0,1848,1.0,1.0,0.0,0,1,0,20,10,discount,5,0,0,1
3,48,3,66.05,52,72000.0,2010,1.0,1.0,1.0,0,1,0,20,10,discount,5,0,0,1
4,108,8,121.86,75,69000.0,246,1.0,1.0,1.0,1,0,0,20,10,discount,5,0,0,1


In [42]:
data_gender.corr()

,gender_F,gender_M,gender_O
gender_F,1.000000,-0.972369,-0.098738
gender_M,-0.972369,1.000000,-0.136299
gender_O,-0.098738,-0.136299,1.000000


In [43]:
aux = float(len(data_gender[data_gender.gender_O == 1])) / float(len(data_gender))
print("{:.2f}%".format(aux*100))

1.37%


___
Since there's high correlation between female and male gender, and O is a small portion of the population we shall keep only the *gender_F* column
___

In [44]:
data = data.drop(columns = ['gender_M','gender_O','offer received'])
data.head()

,recency,frequency,monetary,age,income,days,offer completed,offer viewed,gender_F,difficulty,duration,offer_type,reward,mobile,social,web
0,294,10,29.89,20,30000.0,1316,0.0,0.0,1,20,10,discount,5,0,0,1
1,174,12,320.48,42,96000.0,1142,1.0,1.0,0,20,10,discount,5,0,0,1
2,30,7,118.85,36,56000.0,1848,1.0,0.0,0,20,10,discount,5,0,0,1
3,48,3,66.05,52,72000.0,2010,1.0,1.0,0,20,10,discount,5,0,0,1
4,108,8,121.86,75,69000.0,246,1.0,1.0,1,20,10,discount,5,0,0,1


### 2.2 Data Transformation

___
Divide the data in three datasets, according to the offer_type (so we can create a differente model for each one)
___

In [45]:
bogo = data[data.offer_type == 'bogo'].drop(columns = ['offer_type'])
info = data[data.offer_type == 'informational'].drop(columns = ['offer_type'])
disc = data[data.offer_type == 'discount'].drop(columns = ['offer_type'])

In [46]:
info['offer completed'].unique()

array([0.])

___
As its name suggests, and as we can see in the cell above, offers of type *'Informational'* are never completed, and so they will not be covered my any ML model
___

In [47]:
from sklearn.preprocessing import MinMaxScaler

___
In order to normalize our data we use MinMaxScaler in both datasets **(bogo and disc)**.
___
Split into train and test datasets
___

In [48]:
bogo_y = bogo['offer completed']
bogo_X = bogo.drop(columns = ['offer completed'])
scaler = MinMaxScaler()
bogo_X = pd.DataFrame(scaler.fit_transform(bogo_X), columns = bogo_X.columns,index=bogo_X.index)
bogo_X_train, bogo_X_test, bogo_y_train, bogo_y_test = train_test_split(bogo_X, bogo_y, test_size=0.33, random_state=42)

/home/ec2-user/anaconda3/envs/amazonei_mxnet_p27/lib/python2.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [49]:
disc_y = disc['offer completed']
disc_X = disc.drop(columns = ['offer completed'])
scaler = MinMaxScaler()
disc_X = pd.DataFrame(scaler.fit_transform(disc_X), columns = disc_X.columns,index=disc_X.index)

disc_X_train, disc_X_test, disc_y_train, disc_y_test = train_test_split(disc_X, disc_y, test_size=0.33, random_state=42)

### 2.2 Upload Data to S3 bucket

In [50]:
def make_csv(x, y, filename, data_dir):
    
    # make data dir, if it does not exist
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
    data = np.insert(x,0,y,1)#np.hstack((x, np.transpose(y)))
    df = pd.DataFrame(data)
    df = df.dropna()
    df.to_csv(str(data_dir)+'/'+str(filename),index = False, header = False)
    
    print('Path created: '+str(data_dir)+'/'+str(filename))

In [51]:
data_dir = 'model_data/bogo'
make_csv(bogo_X_train.to_numpy(), bogo_y_train.to_numpy(), filename='train.csv', data_dir=data_dir)
make_csv(bogo_X_test.to_numpy(), bogo_y_test.to_numpy(), filename='test.csv', data_dir=data_dir)

Path created: model_data/bogo/train.csv
Path created: model_data/bogo/test.csv


In [52]:
data_dir = 'model_data/disc'
make_csv(disc_X_train.to_numpy(), disc_y_train.to_numpy(), filename='train.csv', data_dir=data_dir)
make_csv(disc_X_test.to_numpy(), disc_y_test.to_numpy(), filename='test.csv', data_dir=data_dir)

Path created: model_data/disc/train.csv
Path created: model_data/disc/test.csv


___
Upload
___

In [53]:
# should be the name of directory you created to save your features data
bogo_data_dir = 'model_data/bogo'
# set prefix, a descriptive name for a directory  
bogo_prefix = 'bogo'


# upload to S3
bogo_input_data = sagemaker_session.upload_data(path=bogo_data_dir, bucket=bucket, key_prefix=bogo_prefix)
print(bogo_input_data)

s3://sagemaker-us-east-2-341076436662/bogo


In [54]:
# should be the name of directory you created to save your features data
disc_data_dir = 'model_data/disc'
# set prefix, a descriptive name for a directory  
disc_prefix = 'disc'


# upload to S3
disc_input_data = sagemaker_session.upload_data(path=disc_data_dir, bucket=bucket, key_prefix=disc_prefix)
print(disc_input_data)

s3://sagemaker-us-east-2-341076436662/disc


# 3. Model Training

## 3.1 Create two pytorch estimators

___
The source code for the neural network created for this project is in the **source** folder
___
For the estimator we also need to provide the entry point (*train.py*), the output path, role, the instance type for the machine we want to use for our training, the session and some hyperparameters
___

In [55]:
from sagemaker.pytorch import PyTorch

bogo_output_path = 's3://{}/{}'.format(bucket, bogo_prefix)

bogo_estimator = PyTorch(entry_point='train.py',
                    source_dir='source',
                    role=role,
                    framework_version='1.0',
                    train_instance_count=1,
                    train_instance_type='ml.c4.xlarge',
                    output_path = bogo_output_path,
                    sagemaker_session=sagemaker_session,
                    hyperparameters={
                        'input_dim': 14,  
                        'hidden_dim': 20,
                        'output_dim': 1,
                        'epochs': 2000 
                    })

In [56]:
disc_output_path = 's3://{}/{}'.format(bucket, disc_prefix)

disc_estimator = PyTorch(entry_point='train.py',
                    source_dir='source', # this should be just "source" for your code
                    role=role,
                    framework_version='1.0',
                    train_instance_count=1,
                    train_instance_type='ml.c4.xlarge',
                    output_path = disc_output_path,
                    sagemaker_session=sagemaker_session,
                    hyperparameters={
                        'input_dim': 14,  
                        'hidden_dim': 20,
                        'output_dim': 1,
                        'epochs': 2000 
                    })

___
Fit the estimator with the input data
___

In [57]:
%%time 
# train the estimator on S3 training data
bogo_estimator.fit({'train': bogo_input_data})

2021-01-27 17:14:40 Starting - Starting the training job...
2021-01-27 17:14:43 Starting - Launching requested ML instances......
2021-01-27 17:16:05 Starting - Preparing the instances for training......
2021-01-27 17:17:10 Downloading - Downloading input data
2021-01-27 17:17:10 Training - Downloading the training image..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-01-27 17:17:24,680 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2021-01-27 17:17:24,683 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2021-01-27 17:17:24,695 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-01-27 17:17:27,740 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-01-27 17:17:28,047 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py
2021-01-27 17:17:28,047

Epoch: 40, Loss: 0.41216567352949995
Epoch: 41, Loss: 0.4100804616482014
Epoch: 42, Loss: 0.40977426793524174
Epoch: 43, Loss: 0.41177486347995523
Epoch: 44, Loss: 0.41156827797533563
Epoch: 45, Loss: 0.4113858261739482
Epoch: 46, Loss: 0.4092516255349869
Epoch: 47, Loss: 0.4095828220103951
Epoch: 48, Loss: 0.41085626341062464
Epoch: 49, Loss: 0.41424917173604087
Epoch: 50, Loss: 0.40850053712169165
Epoch: 51, Loss: 0.408994309727829
Epoch: 52, Loss: 0.4111654740934715
Epoch: 53, Loss: 0.409325937553725
Epoch: 54, Loss: 0.4105183742167544
Epoch: 55, Loss: 0.41216224360371195
Epoch: 56, Loss: 0.41083340790962614
Epoch: 57, Loss: 0.4123000979918167
Epoch: 58, Loss: 0.409949180645608
Epoch: 59, Loss: 0.4098248140291307
Epoch: 60, Loss: 0.4089256555954945
Epoch: 61, Loss: 0.41087616375167013
Epoch: 62, Loss: 0.40796606669287466
Epoch: 63, Loss: 0.4090832753434138
Epoch: 64, Loss: 0.41060508722709915
Epoch: 65, Loss: 0.40859764820559885
Epoch: 66, Loss: 0.4082792398119204
Epoch: 67, Loss: 0

Epoch: 227, Loss: 0.4035048016332822
Epoch: 228, Loss: 0.40559750780937254
Epoch: 229, Loss: 0.40333364003519545
Epoch: 230, Loss: 0.4025236992152565
Epoch: 231, Loss: 0.40427391862246836
Epoch: 232, Loss: 0.4047705093566148
Epoch: 233, Loss: 0.40590438446130495
Epoch: 234, Loss: 0.40353330971871504
Epoch: 235, Loss: 0.4029094235994756
Epoch: 236, Loss: 0.4037710373646697
Epoch: 237, Loss: 0.40688719617957725
Epoch: 238, Loss: 0.40334890186683603
Epoch: 239, Loss: 0.4016066032666636
Epoch: 240, Loss: 0.4052075764027786
Epoch: 241, Loss: 0.4043573194607206
Epoch: 242, Loss: 0.40418384025288157
Epoch: 243, Loss: 0.4060610382924601
Epoch: 244, Loss: 0.4045746388895383
Epoch: 245, Loss: 0.40399210836691973
Epoch: 246, Loss: 0.4037932030815389
Epoch: 247, Loss: 0.4060378664354813
Epoch: 248, Loss: 0.4031600979353877
Epoch: 249, Loss: 0.40640382743331077
Epoch: 250, Loss: 0.4056029118195609
Epoch: 251, Loss: 0.40357101506155557
Epoch: 252, Loss: 0.40384137110570817
Epoch: 253, Loss: 0.404014

Epoch: 405, Loss: 0.39963428122745015
Epoch: 406, Loss: 0.40030942909306183
Epoch: 407, Loss: 0.4002679765461474
Epoch: 408, Loss: 0.4002812342566244
Epoch: 409, Loss: 0.3995168929917024
Epoch: 410, Loss: 0.3997048097424322
Epoch: 411, Loss: 0.3999682062126459
Epoch: 412, Loss: 0.4024256276505204
Epoch: 413, Loss: 0.40302413176966734
Epoch: 414, Loss: 0.4006646842454891
Epoch: 415, Loss: 0.4042941428087188
Epoch: 416, Loss: 0.3972795270383358
Epoch: 417, Loss: 0.397874460196223
Epoch: 418, Loss: 0.39818573997640677
Epoch: 419, Loss: 0.39691815067489694
Epoch: 420, Loss: 0.3990515795269695
Epoch: 421, Loss: 0.39938308124431765
Epoch: 422, Loss: 0.40119602317364056
Epoch: 423, Loss: 0.40211180288648624
Epoch: 424, Loss: 0.3992255823235871
Epoch: 425, Loss: 0.40008849081780407
Epoch: 426, Loss: 0.4006201432937985
Epoch: 427, Loss: 0.398938333061762
Epoch: 428, Loss: 0.39828521947931617
Epoch: 429, Loss: 0.4041821961271812
Epoch: 430, Loss: 0.40094382366369047
Epoch: 431, Loss: 0.400720645

Epoch: 587, Loss: 0.39782571397917565
Epoch: 588, Loss: 0.39758033978328994
Epoch: 589, Loss: 0.3992521832561839
Epoch: 590, Loss: 0.3989329850700469
Epoch: 591, Loss: 0.39857029433092966
Epoch: 592, Loss: 0.396539156604687
Epoch: 593, Loss: 0.3999828034664091
Epoch: 594, Loss: 0.4017308446196105
Epoch: 595, Loss: 0.4006491381504971
Epoch: 596, Loss: 0.39762143812877837
Epoch: 597, Loss: 0.39885623847877666
Epoch: 598, Loss: 0.39677033083897245
Epoch: 599, Loss: 0.3979435667529558
Epoch: 600, Loss: 0.39889803850304534
Epoch: 601, Loss: 0.39865221424777975
Epoch: 602, Loss: 0.3988278321380808
Epoch: 603, Loss: 0.3959650529978763
Epoch: 604, Loss: 0.39997451278390067
Epoch: 605, Loss: 0.3974768243560034
Epoch: 606, Loss: 0.3982544746069575
Epoch: 607, Loss: 0.39966227944790905
Epoch: 608, Loss: 0.3985416426749727
Epoch: 609, Loss: 0.3972228555624498
Epoch: 610, Loss: 0.3986941662412214
Epoch: 611, Loss: 0.39802705991709214
Epoch: 612, Loss: 0.39888929680278357
Epoch: 613, Loss: 0.3948219

...

Epoch: 1124, Loss: 0.39718910152631015
Epoch: 1125, Loss: 0.39966894379130374
Epoch: 1126, Loss: 0.39769605205575953
Epoch: 1127, Loss: 0.39636252262728755
Epoch: 1128, Loss: 0.3984139272724156
Epoch: 1129, Loss: 0.39639091966248646
Epoch: 1130, Loss: 0.39558864761025414
Epoch: 1131, Loss: 0.39722374205015093
Epoch: 1132, Loss: 0.3957638344411863
Epoch: 1133, Loss: 0.3969037171664492
Epoch: 1134, Loss: 0.39808941749987264
Epoch: 1135, Loss: 0.39792963117876323
Epoch: 1136, Loss: 0.39926131034107626
Epoch: 1137, Loss: 0.39752275555460287
Epoch: 1138, Loss: 0.39616633385399236
Epoch: 1139, Loss: 0.3986795901644197
Epoch: 1140, Loss: 0.3987624269458021
Epoch: 1141, Loss: 0.39744552630379815
Epoch: 1142, Loss: 0.39602427219619063
Epoch: 1143, Loss: 0.3977760084354696
Epoch: 1144, Loss: 0.3952774793311521
Epoch: 1145, Loss: 0.39577265964359826
Epoch: 1146, Loss: 0.3953778959635547
Epoch: 1147, Loss: 0.39734569240624235
Epoch: 1148, Loss: 0.3959561752188915
Epoch: 1149, Loss: 0.3974204087632

Epoch: 1302, Loss: 0.39614957633375464
Epoch: 1303, Loss: 0.39972277187367444
Epoch: 1304, Loss: 0.3941754671599443
Epoch: 1305, Loss: 0.4002616749470989
Epoch: 1306, Loss: 0.3952061203031454
Epoch: 1307, Loss: 0.39523444518982787
Epoch: 1308, Loss: 0.3969976517963088
Epoch: 1309, Loss: 0.3937633116452608
Epoch: 1310, Loss: 0.3994491885739532
Epoch: 1311, Loss: 0.3967924076306375
Epoch: 1312, Loss: 0.3980263013516348
Epoch: 1313, Loss: 0.39476975700450884
Epoch: 1314, Loss: 0.39560247777367363
Epoch: 1315, Loss: 0.4007183493090533
Epoch: 1316, Loss: 0.3979552054254758
Epoch: 1317, Loss: 0.3984550616573826
Epoch: 1318, Loss: 0.39453757281207774
Epoch: 1319, Loss: 0.3953860977388929
Epoch: 1320, Loss: 0.3965589748811425
Epoch: 1321, Loss: 0.3984457387448511
Epoch: 1322, Loss: 0.3953915163050219
Epoch: 1323, Loss: 0.4004133207308718
Epoch: 1324, Loss: 0.39882345913418593
Epoch: 1325, Loss: 0.3967209409350429
Epoch: 1326, Loss: 0.3970922909697639
Epoch: 1327, Loss: 0.3963572896300798
Epoch

Epoch: 1484, Loss: 0.3968488662188066
Epoch: 1485, Loss: 0.40211697666549007
Epoch: 1486, Loss: 0.3991216840802139
Epoch: 1487, Loss: 0.39823057308257204
Epoch: 1488, Loss: 0.3971276013760275
Epoch: 1489, Loss: 0.39923398672792426
Epoch: 1490, Loss: 0.3959610620011185
Epoch: 1491, Loss: 0.3955001760439053
Epoch: 1492, Loss: 0.3972204955730118
Epoch: 1493, Loss: 0.3957023850718145
Epoch: 1494, Loss: 0.4003629828358834
Epoch: 1495, Loss: 0.39660561994686827
Epoch: 1496, Loss: 0.39676572230450013
Epoch: 1497, Loss: 0.3964230375181507
Epoch: 1498, Loss: 0.3990228883561422
Epoch: 1499, Loss: 0.3972804459638863
Epoch: 1500, Loss: 0.3978179348053032
Epoch: 1501, Loss: 0.39751446220565223
Epoch: 1502, Loss: 0.3969766138685689
Epoch: 1503, Loss: 0.3984106775028
Epoch: 1504, Loss: 0.3972029160517545
Epoch: 1505, Loss: 0.396140419060099
Epoch: 1506, Loss: 0.3957019622230266
Epoch: 1507, Loss: 0.3969058182791267
Epoch: 1508, Loss: 0.39644059081810473
Epoch: 1509, Loss: 0.3979823117222982
Epoch: 15

Epoch: 1661, Loss: 0.3966924659237987
Epoch: 1662, Loss: 0.3963902719308562
Epoch: 1663, Loss: 0.3959751349241366
Epoch: 1664, Loss: 0.3964919830655697
Epoch: 1665, Loss: 0.3981222684030134
Epoch: 1666, Loss: 0.39388805374832897
Epoch: 1667, Loss: 0.39566777814514575
Epoch: 1668, Loss: 0.39737464553505225
Epoch: 1669, Loss: 0.3966394367075189
Epoch: 1670, Loss: 0.39974140952937665
Epoch: 1671, Loss: 0.3969770604551333
Epoch: 1672, Loss: 0.3973913657534378
Epoch: 1673, Loss: 0.39707371753281084
Epoch: 1674, Loss: 0.3954779859606491
Epoch: 1675, Loss: 0.39823500920772387
Epoch: 1676, Loss: 0.3967795965891656
Epoch: 1677, Loss: 0.39548666778060493
Epoch: 1678, Loss: 0.3958237018683516
Epoch: 1679, Loss: 0.3953645836618596
Epoch: 1680, Loss: 0.3953374091092688
Epoch: 1681, Loss: 0.39557563401707146
Epoch: 1682, Loss: 0.3934947690961272
Epoch: 1683, Loss: 0.39955725698612043
Epoch: 1684, Loss: 0.3975486573331666
Epoch: 1685, Loss: 0.39835465192939074
Epoch: 1686, Loss: 0.39855494083808335
E

Epoch: 1838, Loss: 0.3993781434415864
Epoch: 1839, Loss: 0.39323511427554353
Epoch: 1840, Loss: 0.39728454201589153
Epoch: 1841, Loss: 0.39741385613219044
Epoch: 1842, Loss: 0.395249122287434
Epoch: 1843, Loss: 0.396322737292053
Epoch: 1844, Loss: 0.39745179156967303
Epoch: 1845, Loss: 0.39664912308832917
Epoch: 1846, Loss: 0.39756542446290144
Epoch: 1847, Loss: 0.4009013477388259
Epoch: 1848, Loss: 0.4000627053296129
Epoch: 1849, Loss: 0.3963331634624453
Epoch: 1850, Loss: 0.3974955464078224
Epoch: 1851, Loss: 0.40088620173898143
Epoch: 1852, Loss: 0.39502156951450546
Epoch: 1853, Loss: 0.3965457345914214
Epoch: 1854, Loss: 0.39559964409528264
Epoch: 1855, Loss: 0.3954518346674998
Epoch: 1856, Loss: 0.3925375807643926
Epoch: 1857, Loss: 0.39990749611760157
Epoch: 1858, Loss: 0.3970579715551265
Epoch: 1859, Loss: 0.3962743823747582
Epoch: 1860, Loss: 0.397021439055272
Epoch: 1861, Loss: 0.3994033807015271
Epoch: 1862, Loss: 0.39825941108548457
Epoch: 1863, Loss: 0.3973855573740655
Epoc

In [58]:
%%time 
# train the estimator on S3 training data
disc_estimator.fit({'train': disc_input_data})

2021-01-27 17:50:16 Starting - Starting the training job...
2021-01-27 17:50:18 Starting - Launching requested ML instances......
2021-01-27 17:51:38 Starting - Preparing the instances for training......
2021-01-27 17:52:45 Downloading - Downloading input data
2021-01-27 17:52:45 Training - Downloading the training image..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-01-27 17:53:00,154 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2021-01-27 17:53:00,157 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2021-01-27 17:53:00,170 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-01-27 17:53:03,214 sagemaker_pytorch_container.training INFO     Invoking user training script.

2021-01-27 17:52:58 Training - Training image download completed. Training in progress.2021-01-27 17:53:03,547 sagemaker-containers INFO   

Epoch: 39, Loss: 0.35293627755706153
Epoch: 40, Loss: 0.354015256069827
Epoch: 41, Loss: 0.3543872133113485
Epoch: 42, Loss: 0.3513237482595427
Epoch: 43, Loss: 0.3501379405976741
Epoch: 44, Loss: 0.351818827168697
Epoch: 45, Loss: 0.3524455156187573
Epoch: 46, Loss: 0.35237842292951943
Epoch: 47, Loss: 0.3528831944908967
Epoch: 48, Loss: 0.3518984532678366
Epoch: 49, Loss: 0.3525814200926428
Epoch: 50, Loss: 0.35010732156970636
Epoch: 51, Loss: 0.34955593532733414
Epoch: 52, Loss: 0.35156193858485935
Epoch: 53, Loss: 0.350815508566097
Epoch: 54, Loss: 0.35055993768664395
Epoch: 55, Loss: 0.3514026367614941
Epoch: 56, Loss: 0.34875697291132
Epoch: 57, Loss: 0.3489467412392861
Epoch: 58, Loss: 0.3505752219391462
Epoch: 59, Loss: 0.35113282814282326
Epoch: 60, Loss: 0.3503577061079813
Epoch: 61, Loss: 0.3500009817379985
Epoch: 62, Loss: 0.3498066772722243
Epoch: 63, Loss: 0.3488134920535331
Epoch: 64, Loss: 0.3496732083766706
Epoch: 65, Loss: 0.3516100863463024
Epoch: 66, Loss: 0.3507533

Epoch: 231, Loss: 0.34481513389072366
Epoch: 232, Loss: 0.3457355252389914
Epoch: 233, Loss: 0.3448189978271682
Epoch: 234, Loss: 0.343599499831007
Epoch: 235, Loss: 0.344717455284351
Epoch: 236, Loss: 0.3459142039231544
Epoch: 237, Loss: 0.3454534843774996
Epoch: 238, Loss: 0.3453963307036056
Epoch: 239, Loss: 0.3449137791610472
Epoch: 240, Loss: 0.3449962551484286
Epoch: 241, Loss: 0.34355057554978113
Epoch: 242, Loss: 0.3461958725180333
Epoch: 243, Loss: 0.34482441074600784
Epoch: 244, Loss: 0.3449285213207795
Epoch: 245, Loss: 0.34586097727766396
Epoch: 246, Loss: 0.34313721507170253
Epoch: 247, Loss: 0.3455131604588328
Epoch: 248, Loss: 0.34476077615525264
Epoch: 249, Loss: 0.3443774355675778
Epoch: 250, Loss: 0.34759789015274517
Epoch: 251, Loss: 0.34450250253266557
Epoch: 252, Loss: 0.34646248111491046
Epoch: 253, Loss: 0.3448702608986413
Epoch: 254, Loss: 0.34711700935914946
Epoch: 255, Loss: 0.3456105870395405
Epoch: 256, Loss: 0.34297710538517623
Epoch: 257, Loss: 0.343916884

Epoch: 408, Loss: 0.34588750625461423
Epoch: 409, Loss: 0.34661662568291907
Epoch: 410, Loss: 0.34343865085959185
Epoch: 411, Loss: 0.3425103067651967
Epoch: 412, Loss: 0.3435721945568137
Epoch: 413, Loss: 0.34322605256855654
Epoch: 414, Loss: 0.34460331155347396
Epoch: 415, Loss: 0.3429750544713676
Epoch: 416, Loss: 0.3449314850516288
Epoch: 417, Loss: 0.3428824334096637
Epoch: 418, Loss: 0.3427554410625023
Epoch: 419, Loss: 0.3438176479157143
Epoch: 420, Loss: 0.3438836394858068
Epoch: 421, Loss: 0.3437404622663409
Epoch: 422, Loss: 0.3440274205933536
Epoch: 423, Loss: 0.34384910182162953
Epoch: 424, Loss: 0.3442029400872715
Epoch: 425, Loss: 0.3436704130189812
Epoch: 426, Loss: 0.34135317848738056
Epoch: 427, Loss: 0.34215302643165485
Epoch: 428, Loss: 0.3421407364596978
Epoch: 429, Loss: 0.3453207226773693
Epoch: 430, Loss: 0.3432962864390394
Epoch: 431, Loss: 0.3428288066106236
Epoch: 432, Loss: 0.3450487589010794
Epoch: 433, Loss: 0.34136894922600597
Epoch: 434, Loss: 0.345017595

Epoch: 585, Loss: 0.34062967299180136
Epoch: 586, Loss: 0.3435357614286573
Epoch: 587, Loss: 0.3428695569308835
Epoch: 588, Loss: 0.34598522650294045
Epoch: 589, Loss: 0.34284729125667673
Epoch: 590, Loss: 0.3444564875925361
Epoch: 591, Loss: 0.34191935882486857
Epoch: 592, Loss: 0.3467713637195157
Epoch: 593, Loss: 0.34117718717309736
Epoch: 594, Loss: 0.3418328488266188
Epoch: 595, Loss: 0.3404204048530214
Epoch: 596, Loss: 0.34172905387372315
Epoch: 597, Loss: 0.3433102836899295
Epoch: 598, Loss: 0.3439561549859708
Epoch: 599, Loss: 0.34147265314469266
Epoch: 600, Loss: 0.34269266296692646
Epoch: 601, Loss: 0.3423536458510638
Epoch: 602, Loss: 0.3439193056814002
Epoch: 603, Loss: 0.3416713422102576
Epoch: 604, Loss: 0.3443850418732846
Epoch: 605, Loss: 0.34207179315532277
Epoch: 606, Loss: 0.3431442129034652
Epoch: 607, Loss: 0.34207515662551463
Epoch: 608, Loss: 0.3405618791967548
Epoch: 609, Loss: 0.34498459566641826
Epoch: 610, Loss: 0.3441428306219245
Epoch: 611, Loss: 0.3430669

...

Epoch: 1127, Loss: 0.3407617583523571
Epoch: 1128, Loss: 0.34301522463169054
Epoch: 1129, Loss: 0.34349118887307895
Epoch: 1130, Loss: 0.3445883972740099
Epoch: 1131, Loss: 0.3439436409601849
Epoch: 1132, Loss: 0.3427827120398413
Epoch: 1133, Loss: 0.3412167243119226
Epoch: 1134, Loss: 0.3428830586204007
Epoch: 1135, Loss: 0.3419035520309387
Epoch: 1136, Loss: 0.34077856977321
Epoch: 1137, Loss: 0.3431916798552426
Epoch: 1138, Loss: 0.34250432894518856
Epoch: 1139, Loss: 0.3396480876581939
Epoch: 1140, Loss: 0.34404513313440166
Epoch: 1141, Loss: 0.34147402224290435
Epoch: 1142, Loss: 0.3432997742099224
Epoch: 1143, Loss: 0.3449518059519279
Epoch: 1144, Loss: 0.34491148659733034
Epoch: 1145, Loss: 0.3436430835792826
Epoch: 1146, Loss: 0.34431180975641745
Epoch: 1147, Loss: 0.34131990786055255
Epoch: 1148, Loss: 0.3419104028205022
Epoch: 1149, Loss: 0.34196826706264805
Epoch: 1150, Loss: 0.3406618828833803
Epoch: 1151, Loss: 0.34209515323389056
Epoch: 1152, Loss: 0.3434630731450013
Epoc

Epoch: 1305, Loss: 0.3424611934692244
Epoch: 1306, Loss: 0.3425994582361032
Epoch: 1307, Loss: 0.34248219734798724
Epoch: 1308, Loss: 0.34212753462260437
Epoch: 1309, Loss: 0.3432726602697776
Epoch: 1310, Loss: 0.34327855176493033
Epoch: 1311, Loss: 0.34095834508198647
Epoch: 1312, Loss: 0.3424564868530763
Epoch: 1313, Loss: 0.34113649136350793
Epoch: 1314, Loss: 0.34475033973132707
Epoch: 1315, Loss: 0.341460168423409
Epoch: 1316, Loss: 0.345045985540797
Epoch: 1317, Loss: 0.34161427599192695
Epoch: 1318, Loss: 0.34191426941456055
Epoch: 1319, Loss: 0.3420422304709807
Epoch: 1320, Loss: 0.3415604798838455
Epoch: 1321, Loss: 0.33985740570505846
Epoch: 1322, Loss: 0.34194587253404585
Epoch: 1323, Loss: 0.34136332596204555
Epoch: 1324, Loss: 0.34060847208171424
Epoch: 1325, Loss: 0.3407927795405916
Epoch: 1326, Loss: 0.34233682486614553
Epoch: 1327, Loss: 0.3433637111833659
Epoch: 1328, Loss: 0.3426303480898457
Epoch: 1329, Loss: 0.34110832079030695
Epoch: 1330, Loss: 0.34263161477473314

Epoch: 1487, Loss: 0.3418536789571218
Epoch: 1488, Loss: 0.34021725553185655
Epoch: 1489, Loss: 0.3430165758624014
Epoch: 1490, Loss: 0.34404870681721944
Epoch: 1491, Loss: 0.34182046198682314
Epoch: 1492, Loss: 0.34339367890298284
Epoch: 1493, Loss: 0.34303098812812516
Epoch: 1494, Loss: 0.34483069955829265
Epoch: 1495, Loss: 0.34204751914572895
Epoch: 1496, Loss: 0.341911009456242
Epoch: 1497, Loss: 0.34249806918142084
Epoch: 1498, Loss: 0.3398183071805751
Epoch: 1499, Loss: 0.34051639695384267
Epoch: 1500, Loss: 0.342144142676824
Epoch: 1501, Loss: 0.3432452740039656
Epoch: 1502, Loss: 0.34244447264521904
Epoch: 1503, Loss: 0.33977490619277756
Epoch: 1504, Loss: 0.34153709790633363
Epoch: 1505, Loss: 0.34128438957673574
Epoch: 1506, Loss: 0.34246252309170794
Epoch: 1507, Loss: 0.34326207095715966
Epoch: 1508, Loss: 0.34260371889795554
Epoch: 1509, Loss: 0.3419831738632316
Epoch: 1510, Loss: 0.3433411370561515
Epoch: 1511, Loss: 0.34124350531325154
Epoch: 1512, Loss: 0.34366568898041

Epoch: 1665, Loss: 0.3441952078407682
Epoch: 1666, Loss: 0.34321666645266896
Epoch: 1667, Loss: 0.3430909779841255
Epoch: 1668, Loss: 0.3447203358702197
Epoch: 1669, Loss: 0.3403200777221821
Epoch: 1670, Loss: 0.3424933838287564
Epoch: 1671, Loss: 0.34545039601922445
Epoch: 1672, Loss: 0.3405795836980707
Epoch: 1673, Loss: 0.34186209179461
Epoch: 1674, Loss: 0.3412942139802305
Epoch: 1675, Loss: 0.34113553554869325
Epoch: 1676, Loss: 0.3437076409524873
Epoch: 1677, Loss: 0.34188075106364585
Epoch: 1678, Loss: 0.34149633409604635
Epoch: 1679, Loss: 0.3410334384495917
Epoch: 1680, Loss: 0.33953677290710954
Epoch: 1681, Loss: 0.33939077041384597
Epoch: 1682, Loss: 0.34182466735847256
Epoch: 1683, Loss: 0.3410741255968521
Epoch: 1684, Loss: 0.34311064662545593
Epoch: 1685, Loss: 0.34050013460939266
Epoch: 1686, Loss: 0.34077284031274896
Epoch: 1687, Loss: 0.33964692850890493
Epoch: 1688, Loss: 0.3434170616934724
Epoch: 1689, Loss: 0.3443450587801635
Epoch: 1690, Loss: 0.3392885854488331
Ep

Epoch: 1847, Loss: 0.34053651541368574
Epoch: 1848, Loss: 0.3424377161392026
Epoch: 1849, Loss: 0.34101201741549686
Epoch: 1850, Loss: 0.3411683358679119
Epoch: 1851, Loss: 0.34324089322851464
Epoch: 1852, Loss: 0.3434071508544955
Epoch: 1853, Loss: 0.3412003440535447
Epoch: 1854, Loss: 0.34351954317152583
Epoch: 1855, Loss: 0.3402511374947733
Epoch: 1856, Loss: 0.34364704000223323
Epoch: 1857, Loss: 0.3438501343187665
Epoch: 1858, Loss: 0.3422681129568395
Epoch: 1859, Loss: 0.3429136935323065
Epoch: 1860, Loss: 0.34073681518620713
Epoch: 1861, Loss: 0.343073976694385
Epoch: 1862, Loss: 0.34223332245659227
Epoch: 1863, Loss: 0.3418110136369768
Epoch: 1864, Loss: 0.3425072081495097
Epoch: 1865, Loss: 0.34130029415567686
Epoch: 1866, Loss: 0.34152530571342915
Epoch: 1867, Loss: 0.3418892437826266
Epoch: 1868, Loss: 0.3424688765730256
Epoch: 1869, Loss: 0.3419560480281422
Epoch: 1870, Loss: 0.34183547141555265
Epoch: 1871, Loss: 0.3406613149438362
Epoch: 1872, Loss: 0.3408909295428318
Epo

# 4. Model Deployment

In [59]:
%%time

from sagemaker.pytorch import PyTorchModel



bogo_model = PyTorchModel(model_data = bogo_estimator.model_data,
                        role = role,
                        framework_version = '1.0',
                        entry_point = 'predict.py',
                        source_dir= 'source')

# deploy and create a predictor
bogo_predictor = bogo_model.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

-------------!CPU times: user 1.1 s, sys: 19 ms, total: 1.12 s
Wall time: 6min 32s


In [60]:
%%time

# deploy your model to create a predictor
disc_model = PyTorchModel(model_data = disc_estimator.model_data,
                        role = role,
                        framework_version = '1.0',
                        entry_point = 'predict.py',
                        source_dir= 'source')

# deploy and create a predictor
disc_predictor = disc_model.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

-------------------!CPU times: user 1.04 s, sys: 33.9 ms, total: 1.08 s
Wall time: 9min 33s


In [61]:
bogo_predictor

In [62]:
disc_predictor

# 5. Model Testing and Results

In [63]:
import os
# read in test data, assuming it is stored locally
bogo_test_data = pd.read_csv(os.path.join(bogo_data_dir, "test.csv"), header=None, names=None)

# labels are in the first column
bogo_test_y = bogo_test_data.iloc[:,0]
bogo_test_x = bogo_test_data.iloc[:,1:]

bogo_test_y_preds = np.squeeze(np.round(bogo_predictor.predict(bogo_test_x)))


assert len(bogo_test_y_preds)==len(bogo_test_y), 'Unexpected number of predictions.'
print('Test passed!')


Test passed!


In [64]:
# read in test data, assuming it is stored locally
disc_test_data = pd.read_csv(os.path.join(disc_data_dir, "test.csv"), header=None, names=None)

# labels are in the first column
disc_test_y = disc_test_data.iloc[:,0]
disc_test_x = disc_test_data.iloc[:,1:]

disc_test_y_preds = np.squeeze(np.round(disc_predictor.predict(disc_test_x)))


assert len(disc_test_y_preds)==len(disc_test_y), 'Unexpected number of predictions.'
print('Test passed!')


Test passed!


## 5.1 Evaluate the Results
- Accuracy
- Precision
- Recall
- F1Score

In [70]:
# code to evaluate the endpoint on test data
# returns a variety of model metrics
def evaluate(predictor, test_features, test_labels, verbose=True):
    """
    Evaluate a model on a test set given the prediction endpoint.  
    Return binary classification metrics.
    :param predictor: A prediction endpoint
    :param test_features: Test features
    :param test_labels: Class labels for test data
    :param verbose: If True, prints a table of all performance metrics
    :return: A dictionary of performance metrics.
    """
    
    # rounding and squeezing array
    test_preds = np.squeeze(np.round(predictor.predict(test_features)))
    
    # calculate true positives, false positives, true negatives, false negatives
    tp = float(np.logical_and(test_labels, test_preds).sum())
    fp = float(np.logical_and(1-test_labels, test_preds).sum())
    tn = float(np.logical_and(1-test_labels, 1-test_preds).sum())
    fn = float(np.logical_and(test_labels, 1-test_preds).sum())
    
    # calculate binary classification metrics
    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    accuracy = (tp + tn) / (tp + fp + tn + fn)
    f1score = tp/(tp+(fp+fn)/2)
        
    return {'TP': tp, 'FP': fp, 'FN': fn, 'TN': tn, 
            'Precision': precision, 'Recall': recall, 'Accuracy': accuracy, 'F1Score': f1score}



In [71]:
bogo_eval = evaluate(bogo_predictor,bogo_test_x, bogo_test_y)

In [72]:
bogo_eval

{'Accuracy': 0.8363049276990032,
 'F1Score': 0.8729293809938972,
 'FN': 356.0,
 'FP': 810.0,
 'Precision': 0.8317757009345794,
 'Recall': 0.9183673469387755,
 'TN': 1952.0,
 'TP': 4005.0}

In [73]:
disc_eval = evaluate(disc_predictor,disc_test_x, disc_test_y)

In [74]:
disc_eval

{'Accuracy': 0.8508698092031426,
 'F1Score': 0.8968261671357857,
 'FN': 256.0,
 'FP': 807.0,
 'Precision': 0.8512990602542841,
 'Recall': 0.9474979491386383,
 'TN': 1445.0,
 'TP': 4620.0}